# IBM Data Science Capstone Project  
## Lab 6: Interactive Dashboard with Plotly Dash
### Project Title: SpaceX Falcon 9 First Stage Landing Prediction  
#### **Assignment**: Interactive Dashboard with Plotly Dash


## TASK: Setup Environment

In [ ]:
!pip install pandas dash plotly
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

## TASK: Read SpaceX dataset

In [ ]:
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

## Create Dash application

In [ ]:
app = dash.Dash(__name__)

## TASK 1: Launch Site Dropdown

In [ ]:
site_options = [{'label': 'All Sites', 'value': 'ALL'}]
site_options += [{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()]

app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard', 
            style={'textAlign': 'center', 'color': '#503D36',
                   'font-size': 40}),
    
    # Dropdown
    dcc.Dropdown(
        id='site-dropdown',
        options=site_options,
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    html.Br(),

## TASK 2: Success Pie Chart

In [ ]:
html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    html.P("Payload range (Kg):"),

## TASK 3: Range Slider

In [ ]:
  dcc.RangeSlider(
        id='payload-slider',
        min=0, max=10000, step=1000,
        marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
        value=[min_payload, max_payload]
    ),

## TASK 4: Scatter Chart

In [ ]:
html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])


## Callback for Pie Chart

In [ ]:
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        fig = px.pie(spacex_df, values='class', 
                     names='Launch Site', 
                     title='Total Successful Launches by Site')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        fig = px.pie(filtered_df, 
                     names='class',
                     title=f"Launch Outcomes for site {selected_site}")
    return fig


## Callback for Scatter Chart

In [ ]:
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [
        Input(component_id='site-dropdown', component_property='value'),
        Input(component_id='payload-slider', component_property='value')
    ]
)
def update_scatter_chart(selected_site, payload_range):
    low, high = payload_range
    mask = (spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)
    filtered_df = spacex_df[mask]
    
    if selected_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]
    
    fig = px.scatter(
        filtered_df, x='Payload Mass (kg)', y='class',
        color='Booster Version Category',
        title='Payload vs. Outcome'
    )
    return fig

## Run app

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True)


### Comlete Lab 6: Interactive Dashboard with Plotly Dash
